In [1]:
import requests,configparser
def get_cookie():
    config = configparser.ConfigParser()
    config.read('secrets.txt')
    cookie = config['session_info']['cookie']
    return cookie
def get_inputs(day):
    cookie, day = get_cookie(), int(day)
    headers = {'session': cookie}
    url = f'https://adventofcode.com/2021/day/{day}/input'
    session = requests.Session()
    resp = session.get(url,cookies=headers)
    return resp.text.split('\n')[:-1]

In [2]:
# DAY1
def count_increases(data,depth):
    increases = 0
    for i in range(depth,len(data)):
        increases += 1 if data[i]>data[i-depth] else 0
    return increases

data_day1 = [int(x) for x in get_inputs(1) if x!='']
print(f'Part 1: {count_increases(data_day1,1)}')
print(f'Part 2: {count_increases(data_day1,3)}')

Part 1: 1482
Part 2: 1518


In [3]:
# DAY2
class Submarine():
    def __init__(self,h_pos,depth,has_aim=False,aim=0):
        self.h_pos = h_pos
        self.depth = depth
        self.has_aim = has_aim
        self.aim = aim
    def forward(self,x):
        self.h_pos += int(x)
        self.depth += int(x)*self.aim
    def down(self,x):
        if self.has_aim:
            self.aim += int(x)
        else:
            self.depth += int(x)
    def up(self,x):
        if self.has_aim:
            self.aim -= int(x)
        else:
            self.depth -= int(x)
    def take_command(self,command):
        command_map = {'up': self.up, 'down':self.down, 'forward':self.forward}
        directions = command.split(' ')    # e.g. direction = ['forward','2']
        move_method = command_map.get(directions[0])
        move_method(directions[1])
    def get_position(self):
        return (self.h_pos,self.depth)
    def navigate(self,data):
        for command in data:
            self.take_command(command)
        return self
    def get_pos_multip(self):
        return self.h_pos*self.depth

data_day2 = get_inputs(2)

d2p1 = Submarine(0,0)
print(f'Part 1: {d2p1.navigate(data_day2).get_pos_multip()}')

d2p2 = Submarine(0,0,True,0)
print(f'Part 2: {d2p2.navigate(data_day2).get_pos_multip()}')


Part 1: 1938402
Part 2: 1947878632


In [4]:
# Day 3
from statistics import mode
data_day3 = get_inputs(3)

def to_matrix(data):
    counters = []
    for l in data:
        counters.append(list(l))
    return counters

def bin_diag(data, idx, most = 1, reduce=False):
    L = len(data[0])
    if len(data) == 1:
        return sum([int(x)*(2**i) for i,x in enumerate(data[0][:idx-L-1:-1])])
    if idx == len(data[0]):
        return 0
    
    ones = [e[idx] for e in data].count('1')
    if ones >= len(data)/2:
        winner = '1' if most else '0'
    else:
        winner = '0' if most else '1'    
    if reduce:
        data = list(filter(lambda x: (x[idx] == winner), data))
    
    return int(winner)*(2**(L-idx-1))+bin_diag(data, idx+1, most, reduce)

def to_decimal(l):
    return sum([int(x)*(2**i) for i,x in enumerate(l[::-1])])

# Part 1
gamma = bin_diag(to_matrix(data_day3), 0, most = 1, reduce=False)
epsilon = bin_diag(to_matrix(data_day3), 0, most = 0, reduce=False)
print(f'Part 1: {gamma*epsilon}')

# Part 2
oxg = bin_diag(to_matrix(data_day3), 0, most = 1, reduce=True)
co2 = bin_diag(to_matrix(data_day3), 0, most = 0, reduce=True)
print(f'Part 2: {oxg*co2}')

Part 1: 3895776
Part 2: 7928162


In [5]:
# Day 4
data_day4 = get_inputs(4)
import re

def solve_day4(data):
    B = Bingo(data)
    # Part 1
    unmarked_totals, last_number = B.find_winnig_grid()
    print(f'Part 1: {unmarked_totals*last_number}')
    
    # Part 2
    unmarked_totals, last_number = B.find_last_winning_grid()
    print(f'Part 2: {unmarked_totals*last_number}')

def get_sequence(data):
    sequence = [int(i) for i in data[0].split(',')]
    return sequence

def get_grids(data): 
    grids = []
    N = len(re.findall(r'[\d]+',data[2]))
    row= 2
    while row < len(data):
        g = [[int(i) for i in re.split('[ ]+',data[j].strip())] for j in range(row,row+5)]
        row += N+1
        grids.append(g)
    return grids

class Bingo():
    def __init__(self,data):
        self.sequence = get_sequence(data)
        self.grids = [Grid(g,self) for g in get_grids(data)]
    
    def find_winnig_grid(self):
        # Find the grid that requires the smallest `winner_idx` in the sequence
        L = min([(idx,grid.find_winner_idx()) for idx,grid in enumerate(self.grids)],key=lambda x:x[1])
        return self.calculate_unmarked_total(L[0],L[1]),self.sequence[L[1]]
    
    def find_last_winning_grid(self):
        # Find the grid that requires the largest `winner_idx` in the sequence
        L = max([(idx,grid.find_winner_idx()) for idx,grid in enumerate(self.grids)],key=lambda x:x[1])
        return self.calculate_unmarked_total(L[0],L[1]),self.sequence[L[1]]
    
    def calculate_unmarked_total(self,grid_idx,seq_idx):
        return self.grids[grid_idx].sum_unmarked(seq_idx)

class Grid():
    def __init__(self,data,B):
        self.grid = data
        self.size = len(data)
        self.B = B
    
    def get_rows(self):    return [(*i,) for i in self.grid]
    def get_row(self,i):   return self.get_rows(i)
    def get_cols(self):    return list(zip(*self.grid))
    def get_col(self,i):   return self.get_cols(i)
    
    def find_winner_idx(self):
        idx = 999
        rows_and_cols = list(set().union(self.get_rows(), self.get_cols()))
        for r in rows_and_cols:
            w = check_for_win(r,self.B.sequence)
            idx = w if (0<w<idx) else idx
        return idx
    
    def sum_unmarked(self,idx):
        s = 0
        for row in self.get_rows():
            s += sum([x for x in row if x not in self.B.sequence[:idx+1]])
        return s
        
def check_for_win(l,sequence):
    matches = [idx for idx, element in enumerate(sequence) if element in l]
    if len(matches)==len(l):
        return max(matches)
    return -1

solve_day4(data_day4)

Part 1: 2745
Part 2: 6594


In [7]:
# Day 5
import operator,collections,re,math
from functools import reduce

def transform_day5(data):
    to_list = [list(map(lambda x:int(x), list(re.findall('(\d+)',i)))) for i in data]
    return list(map(lambda x: list(zip(x[::2],x[1::2])),to_list))

def test_data_day5():
    d5 = '''0,9 -> 5,9
        8,0 -> 0,8
        9,4 -> 3,4
        2,2 -> 2,1
        7,0 -> 7,4
        6,4 -> 2,0
        0,9 -> 2,9
        3,4 -> 1,4
        0,0 -> 8,8
        5,5 -> 8,2
        '''.split('\n')[:-1]
    return d5

class Line():
    def __init__(self,points):
        self.start , self.end = points
    def is_H_or_V(self):
        return any(self.start[i] == self.end[i] for i in range(2))
    def get_points(self):
        points = []
        direction = tuple(0 if self.end[i]==self.start[i] else int(math.copysign(1,self.end[i]-self.start[i])) for i in range(2))
        distance = 1+ max([abs(self.end[i]-self.start[i]) for i in range(2)])
        for d in range(distance):
            new_point = tuple(map(lambda i, j: i + d*j, self.start, direction))
            points.append(new_point)
        return points
        
def solve_day5(data):
    lines = [Line(i) for i in transform_day5(data)]
    
    # Part 1 - horizontal or vertical lines
    all_hv_points = []
    for line in lines:
        if line.is_H_or_V():
            all_hv_points += line.get_points()
    # calculate the frequency of all points
    points_freq = collections.Counter(all_hv_points)
    # print out number of points with frequency more than 1
    print(f'Part 1: {len(list(filter(lambda x: x[1]>1,points_freq.items())))}')
    
    # Part 2 - all lines
    all_points = []
    for line in lines:
        all_points += line.get_points()
    all_points_freq = collections.Counter(all_points)
    print(f'Part 2: {len(list(filter(lambda x: x[1]>1,all_points_freq.items())))}')

solve_day5(get_inputs(5))

Part 1: 5145
Part 2: 16518


In [8]:
# Day 6
import re, collections
test_d6 = ['3,4,3,1,2']

class School():
    def __init__(self,data):
        fish_list = [int(x) for x in re.findall('\d+',data[0])]
        self.fish_dict = self.to_dict(fish_list)
        self.day = 0
    def to_dict(self,f):
        d = collections.Counter(f)
        for i in range(9):
            d[i] = d[i] or 0
        return d
    def one_day(self):
        self.day += 1
        tomorrow_dict = dict(map(lambda x: (x,0),list(range(9))))
        for k,v in self.fish_dict.items():
            if k>=1:
                tomorrow_dict[k-1] = v
        tomorrow_dict[8] += self.fish_dict[0]
        tomorrow_dict[6] += self.fish_dict[0]
        return tomorrow_dict
    def age(self,days):
        for i in range(days):
            self.fish_dict = self.one_day()
        return sum(self.fish_dict.values())
        
s = School(get_inputs(6))
print(f'Part 1: {s.age(80)} lanternfishes on day #{s.day}')
print(f'Part 2: {s.age(256 - 80)} lanternfishes on day #{s.day}')

Part 1: 385391 lanternfishes on day #80
Part 2: 1728611055389 lanternfishes on day #256


In [10]:
# Day 7
import math,re
def transform_day7(data):
    return [int(x) for x in re.findall('\d+',data[0])]

def fuel(x,D):    return sum([abs(i-x) for i in D])
def fuel2(x,D):   return int(sum([(1+abs(i-x))*abs(i-x)/2 for i in D]))
def deriv(x,F,D):   return F(x+1,D)- F(x,D)
def get_d7():     return transform_day7(get_inputs(7))

def search_for_min(D,a,b,F):
    """ This is applying the bisection search method recursively
    to find the 'only' minimum for this continuous function 
    in a closed interval [a,b]"""
    print((a,b))
    if deriv(a,F,D)>0:
        return a
    elif b-a == 1 or deriv(b,F,D)<0:
        return b
    middle = int(math.floor((b+a)/2))
    if deriv(middle,F,D)>0:
        b = middle
    else:
        a = middle
    
    return search_for_min(D,a,b,F)

def solve_day7():
    D = get_d7()
    a, b = min(D), max(D)
    print(f'>> Part 1: {fuel(search_for_min(D,a,b,fuel),D)}')
    print(f'>> Part 2: {fuel2(search_for_min(D,a,b,fuel2),D)}')

solve_day7()

(0, 1927)
(0, 963)
(0, 481)
(240, 481)
(240, 360)
(300, 360)
(330, 360)
(330, 345)
(330, 337)
(330, 333)
(330, 331)
>> Part 1: 349769
(0, 1927)
(0, 963)
(0, 481)
(240, 481)
(360, 481)
(420, 481)
(450, 481)
(465, 481)
(473, 481)
(477, 481)
(477, 479)
(478, 479)
>> Part 2: 99540554


In [95]:
# Day 8

# test data
td8 = """be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb |fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec |fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef |cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega |efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga |gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf |gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf |cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd |ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg |gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc |fgae cfgab fg bagce
""".split('\n')[:-1]

def transform_day8(data):
    return [re.findall('[a-g]+',d) for d in data]

def solve_day8(data):
    d = transform_day8(data)
    # Part 1
    p1 = sum([len([j for j in x[10:] if len(j) in {2,3,4,7}]) for x in d])
    print(f'Part 1: {p1}')
    
    # Part 2
    total = 0
    for t in d:
        s = Signal(t)
        total += s.calculate_output()
    print(f'Part 2: {total}')

def intersect_strs(str1,str2):
    s1, s2 = set(str1), set(str2)
    return len(s1.intersection(s2))

class Signal():
    length_map = [6,2,5,5,4,5,6,3,7,6,6]
    def __init__(self,row):
        self.digits = row[:10]
        self.signal = row[10:]
        self.nums = {}
        self.get_refs()
    def get_refs(self):
        for i in [1,7,4,8]:
            self.nums[i] = list(filter(lambda x: len(x) == self.length_map[i], self.digits))[0]
    def get_number(self,str1):
        if len(str1) in [2,3,4,7]:
            return self.length_map.index(len(str1))
        if len(str1) == 5:
            if intersect_strs(str1,self.nums[1]) == 2:
                return 3
            if intersect_strs(str1,self.nums[4]) == 2:
                return 2
            return 5
        else:
            if intersect_strs(str1,self.nums[7]) == 2:
                return 6
            if intersect_strs(str1,self.nums[4]) == 4:
                return 9
            return 0
    
    def calculate_output(self):
        s = 0
        for i,sig in enumerate(self.signal):
            s+= (10**(3-i))*self.get_number(sig)
        return s
        
    
solve_day8(get_inputs(8))

Part 1: 543
Part 2: 994266


In [91]:
# Day 9
import numpy as np

td9 = '''2199943210
3987894921
9856789892
8767896789
9899965678
'''.split('\n')[:-1]

def transform_day9(data):
    d = [list(i) for i in data]
    d = [[int(i) for i in x] for x in d]
    return d
    # add padding with 10

def insert_padding(d):
    d = [[10 for i in range(len(d[0]))] for j in range(2)]+d+ [[10 for i in range(len(d[0]))]]
    d = [[10,10]+i+[10] for i in d]
    # convert to numpy array
    return np.array(d)
    
def find_dips(n):    
    # first diff
    dh = np.diff(n,n=1)
    dv = np.diff(n, n=1, axis = 0)
    # normalize to -1, 0, 1
    dh = 2*(dh>0)-1+(dh==0)
    dv = 2*(dv>0)-1+(dv==0)
    
    # second diff
    dh = np.diff(dh,n=1)
    dv = np.diff(dv, n=1, axis = 0)
    
    # remove paddings
    dh = np.delete(dh,(0,1,-1),axis = 0)
    dh = np.delete(dh,(0),axis = 1)
    dv = np.delete(dv,(0,1,-1),axis=1)
    dv = np.delete(dv,(0),axis=0)
    
    dh = 2*(dh>0)-1+(dh==0)
    dv = 2*(dv>0)-1+(dv==0)
    
    a = dh+dv
    dip_indices = np.argwhere(a==2)
    
    return dip_indices

class Basin():
    def __init__(self,dip,data):
        self.data = data
        self.dip = dip
        self.points = set()
        self.points.add(tuple(dip))
    
    def expand(self, p = None):    # p=(i,j)
        p = p or self.dip
        D = self.get_dirs(p)
        for d in D:
            new_point = np.add(p,d)
            if tuple(new_point) not in self.points and self.data.item(tuple(new_point)) != 9:
                self.points.add(tuple(new_point))
                self.expand(tuple(new_point))
        return len(self.points)
                
    def get_dirs(self,p):
        dirs = []
        for dim in range(2):
            if p[dim] > 0: # can go UP or LEFT
                dirs.append((-1+dim,-dim))
            if p[dim] < self.data.shape[dim]-1: # can go right or down
                dirs.append((1-dim,0+dim))
        return dirs

def solve_day9(data):
    d = transform_day9(data)
    n = np.array(d)
    
    # Part 1
    padded_d = insert_padding(d)
    dip_indices = find_dips(padded_d)
    risk = 0
    for i in dip_indices:
        risk += n.item(tuple(i)) + 1
    print(f'Part 1: {risk}')
    
    # Part 2
    basin_sizes = [Basin(tuple(i),n).expand() for i in dip_indices]
    basin_sizes.sort(reverse= True)
    top3_multiplied = np.prod(basin_sizes[:3])
    print(f'Part 2: {top3_multiplied}')

solve_day9(get_inputs(9))

Part 1: 535
Part 2: 1122700
